In [1]:
# PySpark 
from pyspark.sql import SparkSession
from pyspark.sql.types import DoubleType
from pyspark.conf import SparkConf
from pyspark.context import SparkContext
# functions
from functions import *
# ML
from pyspark.ml.linalg import Vectors
from pyspark.ml import Pipeline
from pyspark.ml.classification import LinearSVC, RandomForestClassifier, GBTClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.ml.feature import StandardScaler


In [2]:
json_folder = "/Users/cuiyeshuai/Documents/UG modules/Individual Project/provenance-kernel-evaluation-master/datasets/CM-Buildings/*.json"
label_csv = "/Users/cuiyeshuai/Documents/UG modules/Individual Project/provenance-kernel-evaluation-master/datasets/CM-Buildings/graphs.csv"
specific_types_edge = True
specific_types_node = True
level = 4
iri = False
forward = False
qualified_names = {
    "xsd:QName",
    "prov:QUALIFIED_NAME"
}
label_map = {
    "Trusted": "1.0",
    "Uncertain": "0.0"
}
# label_map = {
#     "Valor": 0.0,
#     "Instinct": 1.0,
#     "Mystic": 2.0
# }

In [3]:
conf = SparkConf().setAppName("spark").setMaster("local[*]").set("spark.driver.memory", "15g")
sc = SparkContext(conf=conf)
spark = SparkSession(sc)
# spark.sparkContext.addPyFile("functions.py")

22/04/04 20:34:28 WARN Utils: Your hostname, cuiyeshuaideMacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.31.122 instead (on interface en0)
22/04/04 20:34:28 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/04 20:34:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/04/04 20:34:29 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
# Load the data into rdd (file_path, json_data(string))
file_and_path_rdd = spark.sparkContext.wholeTextFiles(json_folder)

In [5]:
# (file_name, ProvDocument)
document_rdd = file_and_path_rdd.map(lambda x: (x[0].split("/")[-1], (ProvDocument.deserialize(content=x[1]))))

In [6]:
# (file_name, Graphic_encoding_of_ProvDocument)
#encoding_rdd = document_rdd.map(lambda x: (x[0], document_to_encoding(x[1],iri,forward)))
encoding_rdd = file_and_path_rdd.map(lambda x: (x[0].split("/")[-1], json_to_encoding(x[1],iri,forward,qualified_names)))

In [7]:
# (file_name, prov_types of nodes)
if forward:
    types_rdd = encoding_rdd.map(lambda x: (x[0], type_generate(x[1], level, specific_types_node, specific_types_edge)))
    # types_rdd = encoding_rdd.map(lambda x: (x[0], type_generate_mixed(x[1], level, specific_types_node, specific_types_edge)))
else:
    types_rdd = encoding_rdd.map(lambda x: (x[0], type_generate_R(x[1], level, specific_types_node, specific_types_edge)))

In [8]:
# (file_name, prov_types occurence in the graph)
types_count_rdd = types_rdd.map(lambda x: (x[0], count_prov_types(level,x[1])))

In [9]:
# All prov_types in this collection of graphs
all_types = types_count_rdd.flatMap(lambda x: x[1].keys()).distinct().collect()
# Number of distinct prov_types
types_count = len(all_types)
# index_map for prov_types, prov_type -> index
index_map = {all_types[i]: i for i in range(types_count)}
# index -> prov_type
reverse_index_map = {i: all_types[i] for i in range(types_count)}

/Users/cuiyeshuai/Library/Python/3.8/lib/python/site-packages/pyspark/python/lib/pyspark.zip/pyspark/shuffle.py:60: UserWarning: Please install psutil to have better support with spilling
  warnings.warn("Please install psutil to have better "
/Users/cuiyeshuai/Library/Python/3.8/lib/python/site-packages/pyspark/python/lib/pyspark.zip/pyspark/shuffle.py:60: UserWarning: Please install psutil to have better support with spilling
  warnings.warn("Please install psutil to have better "


In [10]:
# Contruct feature vectors for each graph
sparse_matrix_rdd = types_count_rdd.map(lambda x: (x[0], sparse_matrix(x[1], types_count, index_map)))
feature_vector_rdd = sparse_matrix_rdd.map(lambda x: (x[0],Vectors.dense(x[1])))

In [11]:
# Convert to dataframe from rdd
df_features = spark.createDataFrame(feature_vector_rdd).withColumnRenamed("_1", "file").withColumnRenamed("_2", "features")

In [12]:
# Standardize features
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures")
scaler_model = scaler.fit(df_features)
df_features = scaler_model.transform(df_features)

In [13]:
# Change the labels
df_labels = spark.read.csv(label_csv, header=True)
df_labels = df_labels.replace(label_map, subset=["label"])
df_labels = df_labels.withColumn("label", df_labels["label"].cast(DoubleType()))

In [14]:
# Join the features and labels
df = df_features.join(df_labels, df_features.file == df_labels.graph_file).select(df_features.scaledFeatures, df_labels.label).withColumnRenamed("scaledFeatures", "features")

In [15]:
# Split the data into training and testing
train, test = df.randomSplit([0.8, 0.2], seed = 123456)

In [16]:
# Oversample the data
labels = [float(x) for x in label_map.values()]
count = {}
for x in labels:
    count[x] = train.filter(train['label'] == x).count()
maxValue = max(count.values())
ratio = {}
for x in labels:
    ratio[x] = maxValue/count[x]
dataframes = []
for x in labels:
    if(count[x] == maxValue):
        dataframes.append(train.filter(train['label'] == x))
    else:
        dataframes.append(train.filter(train['label'] == x).sample(withReplacement=True, fraction=ratio[x], seed=123456))
train = dataframes[0]
for dataframe in dataframes[1:]:
    train = train.union(dataframe)
print(ratio)
for x in labels:
    print(train.filter(train['label'] == x).count())

{1.0: 1.0, 0.0: 6.405063291139241}


3542


3547


----------------------------------------                                        
Exception happened during processing of request from ('127.0.0.1', 61435)
Traceback (most recent call last):
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.8/lib/python3.8/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.8/lib/python3.8/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.8/lib/python3.8/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.8/lib/python3.8/socketserver.py", line 747, in __init__
    self.handle()
  File "/User

In [ ]:
# LinearSVC classifier
svc = LinearSVC(maxIter = 100, threshold=0.0)

pipeline = Pipeline(stages=[svc])
paramGrid = ParamGridBuilder().addGrid(svc.regParam, [1, 0.1, 0.01,0.001,0.0001,0.00001]).addGrid(svc.maxIter, [100, 500]).build()
# train the model and select the best model using "metricName"(hyperparameter tuning)
crossval = CrossValidator(
    estimator=pipeline, 
    estimatorParamMaps=paramGrid, 
    evaluator=MulticlassClassificationEvaluator(metricName="accuracy"), 
    numFolds=10,
    collectSubModels=True)
cvModel = crossval.fit(train)

# predict the labels of test data
res_test = cvModel.bestModel.transform(test)

# convert to dataframe and compute the metrics
preds_and_labels = res_test.select("prediction", "label").rdd.map(lambda x: (x[0], x[1]))
metrics = MulticlassMetrics(preds_and_labels)
print(metrics.accuracy)
print(metrics.confusionMatrix().toArray())
print(list(zip(cvModel.avgMetrics, paramGrid)))

In [ ]:
coef = cvModel.bestModel.stages[0].coefficients
coef

In [ ]:
# svc = LinearSVC()
# svc_model = svc.fit(train)
# res_test = svc_model.transform(test)

# # convert to dataframe and compute the metrics
# preds_and_labels = res_test.select("prediction", "label").rdd.map(lambda x: (x[0], x[1]))
# metrics = MulticlassMetrics(preds_and_labels)
# print(metrics.accuracy)
# print(metrics.confusionMatrix().toArray())
# svc_model.coefficients

In [ ]:
most_important_features(coef.toArray(), reverse_index_map, True)

In [ ]:
rf = RandomForestClassifier(labelCol="label", featuresCol="features")

pipeline = Pipeline(stages=[rf])
paramGrid = ParamGridBuilder().addGrid(rf.numTrees, [5,10,20,30]).addGrid(rf.maxDepth, [3,4,5,6]).build()
# train the model and select the best model using "metricName"(hyperparameter tuning)
crossval = CrossValidator(
    estimator=pipeline, 
    estimatorParamMaps=paramGrid, 
    evaluator=MulticlassClassificationEvaluator(metricName="accuracy"), 
    numFolds=10,
    collectSubModels=True)
cvModel = crossval.fit(train)

# predict the labels of test data
res_test = cvModel.bestModel.transform(test)

# convert to dataframe and compute the metrics
preds_and_labels = res_test.select("prediction", "label").rdd.map(lambda x: (x[0], x[1]))
metrics = MulticlassMetrics(preds_and_labels)
print(metrics.accuracy)
print(metrics.confusionMatrix().toArray())
print(list(zip(cvModel.avgMetrics, paramGrid)))

In [ ]:
gbt = GBTClassifier(labelCol="label", featuresCol="features")
gbt_model = gbt.fit(train)
res_test = gbt_model.transform(test)

pipeline = Pipeline(stages=[gbt])
paramGrid = ParamGridBuilder().addGrid(gbt.maxIter, [5,10]).addGrid(gbt.maxDepth, [3,4,5,6]).build()
# train the model and select the best model using "metricName"(hyperparameter tuning)
crossval = CrossValidator(
    estimator=pipeline, 
    estimatorParamMaps=paramGrid, 
    evaluator=MulticlassClassificationEvaluator(metricName="accuracy"), 
    numFolds=10,
    collectSubModels=True)
cvModel = crossval.fit(train)

# predict the labels of test data
res_test = cvModel.bestModel.transform(test)

# convert to dataframe and compute the metrics
preds_and_labels = res_test.select("prediction", "label").rdd.map(lambda x: (x[0], x[1]))
metrics = MulticlassMetrics(preds_and_labels)
print(metrics.accuracy)
print(metrics.confusionMatrix().toArray())
print(list(zip(cvModel.avgMetrics, paramGrid)))

In [ ]:
df_features = spark.createDataFrame(sparse_matrix_rdd).withColumnRenamed("_1", "file").withColumnRenamed("_2", "features")
df = df_features.join(df_labels, df_features.file == df_labels.graph_file).select(df_features.features, df_labels.label)
df_list = df.collect()


In [ ]:
train_list = train.collect()
test_list = test.collect()
X_train = [x[0] for x in train_list]
y_train = [x[1] for x in train_list]
X_test = [x[0] for x in test_list]
y_test = [x[1] for x in test_list]

In [ ]:
import numpy as np
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score

# X = np.array([x[0] for x in df_list])
# y = np.array([x[1] for x in df_list])
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
clf = make_pipeline(StandardScaler(),
    SVC(kernel="linear"))
clf.fit(X_train, y_train)
(clf.predict(X_test), y_test)
print(confusion_matrix(clf.predict(X_test), y_test))
print(accuracy_score(clf.predict(X_test), y_test))

In [ ]:
from sklearn.svm import LinearSVC

# X = np.array([x[0] for x in df_list])
# y = np.array([x[1] for x in df_list])
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
clf = make_pipeline(StandardScaler(),
    LinearSVC())
clf.fit(X_train, y_train)
(clf.predict(X_test), y_test)
print(confusion_matrix(clf.predict(X_test), y_test))
print(accuracy_score(clf.predict(X_test), y_test))